
<br>
PyAudio Example: Play a wave file (callback version).<br>
Almost constant delay = 0.026s +- 0.002s<br>


In [1]:
import librosa
from scipy import signal
import numpy as np
import pyaudio, time
# import wave
import sys, pdb 

In [5]:
WAVE_OUTPUT_FILENAME = "C:/Users/Dell/OneDrive/Desktop/audiofiles/song.wav"
RATE = 16000

In [7]:
y_play, _ = librosa.load('C:/Users/Dell/OneDrive/Desktop/audiofiles/song.wav',sr=RATE)

RECORD_SECONDS = len(y_play)/RATE

In [12]:
import soundfile as sf

In [13]:
if True:
    CHUNK = 128
    # WIDTH = 2
    CHANNELS = 1
    p = pyaudio.PyAudio()
    y_record = []
    i=0
    def callback(in_data, frame_count, time_info, status):
        global y_record, i, y_play

        # REC
        y_record.append(in_data)

        # PLAY
        data = y_play[i*frame_count:(i+1)*frame_count]
        i+=1
        data = data.astype(np.float32).tostring()
        return (data, pyaudio.paContinue)
    stream = p.open(format=pyaudio.paFloat32, #p.get_format_from_width(WIDTH),
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    output=True,
                    frames_per_buffer=CHUNK,
                    stream_callback=callback)
    print("* recording")
    stream.start_stream()
    while stream.is_active():
        time.sleep(0.01)
    stream.stop_stream()
    stream.close()
    p.terminate()
    y_record = [np.frombuffer(data, dtype=np.float32) for data in y_record]
    y_record = np.concatenate(y_record)
    sf.write(WAVE_OUTPUT_FILENAME, y_record, RATE)

* recording


C:\Users\Dell\AppData\Local\Temp/ipykernel_4004/2999226467.py:17: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  data = data.astype(np.float32).tostring()


In [14]:
def delay_core(reference, recorded, DEBUG=False):
        sr=RATE
        corr = signal.correlate(reference, recorded, mode='full') 
        d = -(np.argmax(corr) - len(recorded) + 1)
        return d       

In [15]:
def delay_audio(y_play, y_record):
    delay_in_sec = delay_core(y_play, y_record, DEBUG=True) / RATE
    delay_in_sec = str(abs(np.round(delay_in_sec, 3)))
    return delay_in_sec

In [16]:
delay = delay_audio(y_play, y_record)
print('Delay',delay)
print('Buffer',CHUNK/RATE)

Delay 0.281
Buffer 0.008
